In [1]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import get_as_dataframe, set_with_dataframe
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import time

#### Load Google credentials and project settings with the gspread module 

Before you can use the module, you need to obtain OAuth2 credentials from Google ("http://gspread.readthedocs.io/en/latest/oauth2.html"). Save the json file with credentials to the local disk. Also remember to share the spreadsheet with the client_email listed in the json file.

In [2]:
scope = ['https://spreadsheets.google.com/feeds']
credentials = ServiceAccountCredentials.from_json_keyfile_name('My Project-cd8f0fca74a6.json', scope)
gc = gspread.authorize(credentials)

In [3]:
# Load power outage spreadsheet and save as pandas dataframe

In [4]:
wks = gc.open("Massachusetts power outage data").sheet1
outages_df = get_as_dataframe(wks)

In [5]:
# Save the list of cities and towns in Massachusetts as a list

In [6]:
municipalities = outages_df['Municipality'].unique()

#### Scrape power outage reports from the MEMA website ("http://mema.mapsonline.net/public.html")

In [7]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import re
from __future__ import division

In [8]:
# Launch a browser with selenium and navigate to power outage reports

In [9]:
driver = webdriver.Chrome()
driver.get("http://mema.mapsonline.net/public.html")

In [10]:
outagereport = driver.find_element_by_xpath('/html/body/div[7]/div/div[3]/div')
outagereport.click()
time.sleep(10)

In [11]:
# Parse the HTML with BeautifulSoup

In [12]:
html_source = driver.page_source
mema_soup = BeautifulSoup(html_source)

In [13]:
mema_table = mema_soup.find("div", class_="muni_table")
mema_rows = mema_table.findAll(lambda tag: tag.name=='tr')
mema_rows.pop(0)

<tr class="tablesorter-headerRow" role="row"><th aria-controls="table_503879" aria-disabled="false" aria-label=": No sort applied, activate to apply an ascending sort" aria-sort="none" class="tablesorter-header tablesorter-headerUnSorted" data-column="0" role="columnheader" scope="col" style="user-select: none;" tabindex="0" unselectable="on"><div class="tablesorter-header-inner"><img src="/mema/poweroutage/1_town.png"/></div></th><th aria-controls="table_503879" aria-disabled="false" aria-label=": Descending sort applied, activate to apply an ascending sort" aria-sort="descending" class="tablesorter-header tablesorter-headerDesc" data-column="1" role="columnheader" scope="col" style="user-select: none;" tabindex="0" unselectable="on"><div class="tablesorter-header-inner"><img src="/mema/poweroutage/3_pct_no_power.png"/></div></th><th aria-controls="table_503879" aria-disabled="false" aria-label=": No sort applied, activate to apply an ascending sort" aria-sort="none" class="tablesorte

In [14]:
# Save the data from each city and town to a dictionary (mema_outages)

In [15]:
mema_outages = {}
for town in mema_rows:
    townoutages = town.findAll('td')
    town_name = townoutages[0].text
    town_name_matched = process.extract(town_name, municipalities, limit=1)
    townoutages_dict = {'customers_served': int(townoutages[3].text),
                        'customers_affected': int(townoutages[2].text),
                        'percent_of_total': float(townoutages[1].text.replace("%", "")),
                        'percent_of_total_string': townoutages[1].text}
    mema_outages[town_name_matched[0][0]] = townoutages_dict

#### Update the Google spreadsheet

In [16]:
outages_df['% without power'] = "0%"
outages_df['% without power numeric'] = "0.0"
outages_df['Number of outages'] = "0"
outages_df['Total number of customers'] = ""

In [17]:
outages_df = outages_df[~outages_df['Municipality'].isin(mema_outages.keys())]

In [18]:
for key, value in mema_outages.iteritems():
    outages_df = outages_df.append({'Municipality': key,
                                    '% without power': value['percent_of_total_string'],
                                    '% without power numeric': value['percent_of_total'],
                                    'Number of outages': value['customers_affected'],
                                    'Total number of customers': value['customers_served']}, ignore_index=True)

In [19]:
outages_df = outages_df.sort_values(by='Municipality', ascending=True)

In [20]:
set_with_dataframe(wks, outages_df)

In [21]:
# Save to CSV file on local disk

In [22]:
outages_df.to_csv("outages_df.csv", index=False)